## Prerequisites
- Nerlnet 1.6.0 is built (`./NerlnetBuild.sh --infra torch`).
- Synthetic CSV is synced to `/tmp/nerlnet/data/NerlnetData-master/nerlnet/synthetic_norm/synthetic_full.csv`.
- The example JSONs (`exp_torch_s1_w4.json`, `dc_torch_s1_w4.json`, `conn_torch_s1_w4.json`) remain inside `examples/torch/s1_w4_syntetic`.

## 1. Generate the TorchScript model
The following cell recreates the placeholder perceptron that matches the JSON metadata and writes it to `examples/torch/s1_w4_syntetic/models/synthetic_torch_example.pt`.

In [ ]:
from pathlib import Path
from typing import Sequence

import torch

MODEL_PATH = Path("examples/torch/s1_w4_syntetic/models/synthetic_torch_example.pt").resolve()
INPUT_SIZE = 5
HIDDEN_SIZES: Sequence[int] = (30, 5)
LABELS = 3
SAMPLES = 2000
EPOCHS = 400
LEARNING_RATE = 0.005
SEED = 2025


def generate_dataset(num_samples: int, num_features: int, num_labels: int, seed: int) -> tuple[torch.Tensor, torch.Tensor]:
    generator = torch.Generator().manual_seed(seed)
    features = torch.rand((num_samples, num_features), generator=generator) * 10.0
    weight = torch.randn((num_features, num_labels), generator=generator)
    bias = torch.randn(num_labels, generator=generator)
    logits = features @ weight + bias
    labels = torch.softmax(logits, dim=1)
    return features, labels


def build_model(input_size: int, hidden_sizes: Sequence[int], output_size: int) -> torch.nn.Module:
    layers = []
    in_dim = input_size
    for hidden in hidden_sizes:
        layers.append(torch.nn.Linear(in_dim, hidden))
        layers.append(torch.nn.ReLU())
        in_dim = hidden
    layers.append(torch.nn.Linear(in_dim, output_size))
    return torch.nn.Sequential(*layers)


class FeatureSliceModule(torch.nn.Module):
    def __init__(self, backbone: torch.nn.Module, feature_width: int) -> None:
        super().__init__()
        self.backbone = backbone
        self.feature_width = feature_width

    def forward(self, inputs: torch.Tensor) -> torch.Tensor:
        if inputs.size(-1) < self.feature_width:
            raise RuntimeError(
                f"Expected at least {self.feature_width} columns, received {inputs.size(-1)}"
            )
        features_only = inputs[..., : self.feature_width]
        return self.backbone(features_only)


def train_model(model: torch.nn.Module, features: torch.Tensor, labels: torch.Tensor, epochs: int, lr: float) -> None:
    criterion = torch.nn.MSELoss()
    for _ in range(max(1, epochs)):
        preds = model(features)
        loss = criterion(preds, labels)
        loss.backward()
        with torch.no_grad():
            for param in model.parameters():
                if param.grad is None:
                    continue
                param -= lr * param.grad
                param.grad.zero_()


torch.manual_seed(SEED)
MODEL_PATH.parent.mkdir(parents=True, exist_ok=True)
features, labels = generate_dataset(SAMPLES, INPUT_SIZE, LABELS, SEED)
model = build_model(INPUT_SIZE, HIDDEN_SIZES, LABELS)
train_model(model, features, labels, EPOCHS, LEARNING_RATE)
model.eval()
scripted = torch.jit.script(FeatureSliceModule(model, INPUT_SIZE))
scripted.save(str(MODEL_PATH))
print(f"[TorchS1W4] Wrote TorchScript model to {MODEL_PATH}")


## 2. Inspect the distributed configuration
This helper cell visualizes the Torch worker definition and confirms the per-batch metadata that the worker expects during training.

In [ ]:
import json
from pprint import pprint

with open("examples/torch/s1_w4_syntetic/dc_torch_s1_w4.json", "r", encoding="utf-8") as handle:
    dc = json.load(handle)
workers = dc["workers"]
train_params = dc["model_sha"]["torch_s1w4_v160"]["train_params"]
print("Workers bound to the example model:")
pprint(workers)
print("\nTorch training parameters:")
pprint(train_params)


## 3. Stage the JSONs and launch Nerlnet
1. Copy or symlink the JSON files into the paths referenced by `config/jsonsDir.nerlconfig` and `config/subnets.nerlconfig` (for example, point `dc.json` to `examples/torch/s1_w4_syntetic/dc_torch_s1_w4.json`).
2. Ensure `conn.json` resolves to `examples/torch/s1_w4_syntetic/conn_torch_s1_w4.json` and the experiment description is reachable at `examples/torch/s1_w4_syntetic/exp_torch_s1_w4.json`.
3. Build or rebuild the native bridges if anything changed (`./NerlnetBuild.sh --infra all`).
4. Launch the Erlang application via `./NerlnetRun.sh` in one terminal and keep it running.

## 4. Kick off the Torch full-flow
With Nerlnet running, trigger the example from a second terminal:
```bash
tests/NerlnetFullFlowTorchTest.sh --manual-start \
  --experiment examples/torch/s1_w4_syntetic/exp_torch_s1_w4.json \
  --dc examples/torch/s1_w4_syntetic/dc_torch_s1_w4.json \
  --conn examples/torch/s1_w4_syntetic/conn_torch_s1_w4.json
```
As batches stream through `s1`, each worker (`w1`-`w4`) stays active on the Torch bridge, preserving optimizer momentum and weights between messages.
Once both training and prediction phases finish, inspect `/tmp/nerlnet/results/` for anomaly summaries and `/tmp/nerlnet/torch/models/pt/` for updated checkpoints.

## 5. Analyze outcomes
Review `test_torch.log` (or the CSVs saved under `/tmp/nerlnet/results/.../model_perf.csv`) to compare Torch metrics against the OpenNN baseline. If the anomaly scores diverge, double-check that the generated TorchScript file matches the shapes advertised in `train_params` and that the JSON paths inside `config/jsonsDir.nerlconfig` are up to date.